In [46]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from collections import defaultdict
import json
from scipy.stats import norm
import matplotlib.pyplot as plt

In [ ]:
file_path = '../../data/race_data/2018.csv'
df = pd.read_csv(file_path)

new_times_lap = []
for index, time in df['Lap Time'].iteritems():
    if time.count(':') == 1:
        new_times_lap.append(datetime.strptime(time, '%M:%S.%f'))
    else:
        new_times_lap.append(datetime.strptime(time, '%H:%M:%S.%f'))
df['Lap Time'] = new_times_lap

new_times_s = []
new_day = False
for index, time in df['Session Time'].iteritems():
    if time.count(':') == 0:
        new_day = True
        if float(time) < 10: 
            new_time = '01:00:00:0' + time
        else:
            new_time = '01:00:00:' + time
        new_date = datetime.strptime(new_time, '%d:%H:%M:%S.%f')
        new_date_plus = new_date + timedelta(days=1)
        new_times_s.append(new_date_plus)

    elif time.count(':') == 1:
        new_date_plus = datetime.strptime(time, '%M:%S.%f')
        if new_day: 
            new_date_plus = new_date_plus + timedelta(days=1)

        new_times_s.append(new_date_plus)

    else:
        new_times_s.append(datetime.strptime(time, '%H:%M:%S.%f'))

df['Session Time'] = new_times_s

df['Prev Session Time'] = df['Session Time'].subtract(df['Lap Time'])

for i in range(1,13):
    num_str = str(i)
    if i < 10: num_str = '0' + num_str
    num_str = 'S' + num_str

    vals = []
    for ind, split_time in df[num_str].iteritems():
        vals.append(timedelta(seconds=split_time))
    df[num_str] = vals
    totals = df[num_str] + df['Prev Session Time']

    for j in range(i-1, 0, -1):
        new_num_str = str(j)
        if j < 10: new_num_str = '0' + new_num_str
        new_num_str = 'S' + new_num_str
        totals += df[new_num_str]
    df[num_str+'totals'] = totals

df.to_csv('../../data/2018/2018_mod.csv', index=False, date_format='%Y-%m-%d %H:%M:%S.%f')

In [ ]:
section_mod = defaultdict(list)
for index, row in df.iterrows():
    section_mod[row['Driver']].append((1, row['Prev Session Time'].total_seconds(), row['S01totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((2, row['S01totals'].total_seconds(), row['S02totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((3, row['S02totals'].total_seconds(), row['S03totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((4, row['S03totals'].total_seconds(), row['S04totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((5, row['S04totals'].total_seconds(), row['S05totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((6, row['S05totals'].total_seconds(), row['S06totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((7, row['S06totals'].total_seconds(), row['S07totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((8, row['S07totals'].total_seconds(), row['S08totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((9, row['S08totals'].total_seconds(), row['S09totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((10, row['S09totals'].total_seconds(), row['S10totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((11, row['S10totals'].total_seconds(), row['S11totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((12, row['S11totals'].total_seconds(), row['S12totals'].total_seconds(), row.Car, row.SPI))

In [ ]:
MAXTIME = df['S12totals'].max().total_seconds()
def roundTo(x, base=20):
    return base * round(x/base)
MAXTIME = roundTo(MAXTIME, 20)

In [ ]:
pit_times = {}
for car in list(pd.unique(df.Car)):
    df_car = df.loc[(df['Car'] == car) & (df['Location'] == 'Pit')]
    pt = list(pd.unique(df_car['Session Time']))
    d_zero = np.datetime64('1900-01-01T00:00:00')
    pt = [(d - d_zero) / np.timedelta64(1, 's') for d in pt]
    pt = [0] + pt
    pit_times[car] = pt

In [ ]:
def getLastPit(car, time):
    lp = max([t for t in pit_times[car] if t <= time])
    return (time - lp) / 60

avg_pit = {}
for k, v in pit_times.items():
    diffs = [j-i for i, j in zip(v[:-1], v[1:])]
    avg_pit[k] = np.average(diffs) / 60
    
aplist = [k for k,v in sorted(avg_pit.items(), key=lambda t: t[1], reverse=True)]
pos_dict = {}
for i in range(len(aplist)):
    pos_dict[aplist[i]] = i+1

print(pos_dict[38], avg_pit[38], getLastPit(38, 8960))
print(pos_dict[52], avg_pit[52], getLastPit(52, 35540))
print(pos_dict[73], avg_pit[73], getLastPit(73, 54240))

In [ ]:
cars = [38, 52, 73]
times = [8960, 35540, 54240]
stopped_pit = [(t[0], t[1], getLastPit(t[0], t[1])) for t in zip(cars, times)]

with open('../../data/2018/18_stopped_car_pit.csv', 'w') as f:
    f.write('Car,Flag Time,Last Pit\n')
    for s in stopped_pit:
        f.write("%s,%s,%s\n"%(s[0], s[1], s[2]))

In [ ]:
def get_risk_score(x):
    alpha = 4
    beta = 1.3
    MEAN1 = 0
    MEAN2 = 45.25483333333322
    STD1 = 15.610617771584481
    STD2 = 9.909181352955741
    return (alpha*norm.pdf(x, MEAN1, STD1) + beta*norm.pdf(x, MEAN2, STD2))*10

In [ ]:
# risk_score = {}
# for t in range(0, MAXTIME, 600):
#     agg_risk = 0
#     for car in pit_times:
#         lp = getLastPit(car, t)
#         agg_risk += get_risk_score(lp)
#     risk_score[t] = agg_risk

In [ ]:
risk_score = {}
sus_score = {}
for t in range(0, MAXTIME, 300):
    agg_risk = 0
    agg_sus = 0
    for car in pit_times:
        lp = getLastPit(car, t)
        
        previous = [time for time in pit_times[car] if time <= t]
        pt = 0
        sus = 0
        for p in previous[1:]:
            if p - pt < 600:
#                 print(previous)
                sus += 100
            elif p - pt < 900:
                sus += 80
            elif p - pt < 1500:
                sus += 30
            else:
                sus -= .25
            pt = p
        agg_sus += sus
        
        agg_risk += get_risk_score(lp) 
    risk_score[t] = agg_risk
    sus_score[t] = agg_sus

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(list(risk_score.keys()), list(risk_score.values()))
plt.vlines(times, 0, 50)
plt.show()

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(list(sus_score.keys())[:95], list(sus_score.values())[:95])
plt.vlines(times, -10000, 1000)
plt.show()

In [ ]:
sus_deriv = {}
for t in range(300, MAXTIME, 300):
    sus_deriv[t] = (sus_score[t] - sus_score[t-300]) / 300
plt.figure(figsize=(15,7))
plt.plot(list(sus_deriv.keys()), list(sus_deriv.values()))
plt.vlines(times, 0, 1)
plt.show()

In [ ]:
with open('../../data/2018/18_risk_score.csv', 'w') as f:
    f.write('Time,Risk Score\n')
    for k,v in risk_score.items():
        f.write('%s,%s\n'%(k,v))

In [ ]:
with open('../../data/2018/18_sus_deriv.csv', 'w') as f:
    f.write('Time,Sus Deriv\n')
    for k,v in sus_deriv.items():
        f.write('%s,%s\n'%(k,v))

In [6]:
distances = {1: 1876, 2: 1751.33, 3: 1768.67, 4: 2352, 5: 1488, 6: 1475, 7: 1427, 
             8: 1428, 9: 1457, 10: 1484, 11: 1283, 12: 1006.83}

num_drivers = defaultdict(lambda: defaultdict(list))
concentration = defaultdict(lambda: defaultdict(float))
last_pit = {}
avg_speed = {}
for t in range(20, MAXTIME, 20):
    cars = []
    speeds = []
    for driver in section_mod:
        for interval in section_mod[driver]:
            if interval[1] <= t <= interval[2]:
                num_drivers[t][interval[0]].append((interval[1], interval[2]))
                cars.append(interval[3])
                speeds.append(interval[4])
                break
    for section in num_drivers[t]:
        concentration[t][section] = (len(num_drivers[t][section]) / distances[section]) * 1000
        # cars per 1000 feet
    
    all_times = []
    for car in cars:
        lp = max(filter(lambda pt: pt < t, pit_times[car]))
        all_times.append(t - lp)
    last_pit[t] = np.max(all_times) / 60 if len(all_times) > 0 else -1
    avg_speed[t] = np.average(speeds)

C:\Users\dithu\anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\dithu\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [9]:
with open('../../data/2018/num_drivers.json', 'w') as outfile:
    json.dump(num_drivers, outfile)
with open('../../data/2018/18_concentrations.csv', 'w') as f:
    f.write('Time,Sector,Concentration\n')
    for time in concentration.keys():
        for sector in range(1,14):
            f.write("%s,%s,%s\n"%(time, sector, concentration[time].get(sector, 0)))

In [10]:
with open('../../data/2018/time_since_pit.csv', 'w') as f:
    f.write('Time,Last Pit\n')
    for time, lp in last_pit.items():
        f.write("%s,%s\n"%(time, lp))

In [11]:
with open('../../data/2018/18_avg_speed.csv', 'w') as f:
    f.write('Time,Speed\n')
    for time, speed in avg_speed.items():
        f.write("%s,%s\n"%(time, speed))